# VishwamAI Math Integration
This notebook implements math problem generation and solving capabilities using VishwamAI architecture.

In [1]:
import sys
sys.path.append('..')

import torch
import pandas as pd
from vishwamai.model import VishwamaiModel, VishwamaiConfig
from vishwamai.architecture import init_model
from vishwamai.conceptual_tokenizer import ConceptualTokenizer
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Initialize model configuration
config = VishwamaiConfig(
    vocab_size=32000,
    hidden_size=4096,
    num_hidden_layers=32,
    num_attention_heads=32,
    num_key_value_heads=8,
    intermediate_size=16384,
    max_position_embeddings=8192
)

# Initialize model and tokenizer
model = init_model(config)
tokenizer = ConceptualTokenizer()

In [ ]:
# Load datasets
def load_datasets():
    # Load regular GSM8K dataset
    train_data = pd.read_parquet('gsm8k/train-00000-of-00001.parquet')
    test_data = pd.read_parquet('gsm8k/test-00000-of-00001.parquet')
    
    # Load Socratic version
    socratic_train = pd.read_parquet('gsm8k/socratic/train-00000-of-00001.parquet')
    socratic_test = pd.read_parquet('gsm8k/socratic/test-00000-of-00001.parquet')
    
    return train_data, test_data, socratic_train, socratic_test

train_data, test_data, socratic_train, socratic_test = load_datasets()

In [ ]:
class MathAI:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        
    def generate_text(self, prompt, max_length=200, temperature=0.7):
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                temperature=temperature,
                do_sample=True
            )
            
        return self.tokenizer.decode(outputs[0])
    
    def generate_problem(self, topic, difficulty):
        prompt = f"Generate a {difficulty} math problem about {topic}:\n"
        return self.generate_text(prompt)
    
    def solve_problem(self, problem):
        prompt = f"Solve this math problem step by step:\n{problem}\n\nSolution:"
        return self.generate_text(prompt, max_length=500, temperature=0.3)
    
    def socratic_solve(self, problem):
        prompt = f"Break down and solve this problem using the Socratic method:\n{problem}"
        return self.generate_text(prompt, max_length=1000, temperature=0.3)

# Initialize MathAI
math_ai = MathAI(model, tokenizer)

In [ ]:
# Training loop
def train_epoch(model, optimizer, train_dataloader, device):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs.view(-1, outputs.size(-1)), labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_dataloader)

def evaluate(model, test_dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_dataloader:
            inputs = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(inputs)
            loss = torch.nn.functional.cross_entropy(outputs.view(-1, outputs.size(-1)), labels.view(-1))
            total_loss += loss.item()
    return total_loss / len(test_dataloader)

In [ ]:
# Training visualization
def plot_training_progress(train_losses, test_losses):
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(test_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Progress')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Example usage
# Generate a problem
problem = math_ai.generate_problem("algebra", "intermediate")
print("Generated Problem:\n", problem)

# Solve using standard approach
solution = math_ai.solve_problem(problem)
print("\nStandard Solution:\n", solution)

# Solve using Socratic method
socratic_solution = math_ai.socratic_solve(problem)
print("\nSocratic Solution:\n", socratic_solution)